## CTR

In [1]:
#coding=utf-8
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack
from sklearn import metrics
from sklearn.externals import joblib
from sklearn.preprocessing.data import OneHotEncoder
import json
import xgboost as xgb
import time
import multiprocessing as mp
import os
os.environ["OMP_NUM_THREADS"] = "6"  #并行训练
import lightgbm 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed


/Users/wjj/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/Users/wjj/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/wjj/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the

In [2]:
def _load_data(data_file,feature_list):
    data_df = pd.read_csv(filepath_or_buffer = data_file,sep=",", header=None, 
                              names=feature_list)
    return data_df

In [3]:
goods_file = '/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/data/1212/goods_feature_1.csv'
goods_feature_list = ['goods_slug', '1', '2','4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', 'EA1', 'EA2', 'EA3', 'EA4', 'EA5', 'EA6', 'EA7', 'EA8', 'EA9', 'EA10', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'j', 'k', 'x', 'y', 'z', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9', 'class_1', 'p_7394376616792', 'p_7394376616815', 'p_ea10days_1512g', 'p_ea_3daypttp', 'p_easyace_ysjnew_xmy_190828', 'p_easyfun_madai_tea_20', 'p_ef_funsichi_nwlb', 'p_ef_zkxsd_757g', 's_1upbar_20190308', 's_ES_20190724', 's_bh_BH20F_mini_charge', 's_bu_BH20SW_white', 's_cm25_green_coffee_140ml', 's_cm25_gxmydr_300g', 's_cm25_shucai_50g', 's_cm25_yyb_384g', 's_cm25_yykkxin_420g_new_190103', 's_coffee_190819_1_1', 's_ea_dcnx_240g', 's_ea_xckfy_350g', 's_ea_ximeiyin_210ml', 's_ea_yyrtg_36g', 's_ea_yyyxcw_420g', 's_easyace_kfdby_400g', 's_easyace_ysj_new_28g', 's_easyfun_beefdoupu_150g', 's_easyfun_beefrice_160g', 's_easyfun_blackrice_xjyz_zs_qyz', 's_easyfun_chia_120g_180604', 's_easyfun_chiaoat_300g', 's_easyfun_cold_coffee_30g', 's_easyfun_curryrice_160g', 's_easyfun_dizhijixiongrou_100g', 's_easyfun_dongyingong_350g', 's_easyfun_geshidongka_100g', 's_easyfun_guodong_new_360g', 's_easyfun_heimiquan_240g', 's_easyfun_jrw_new_120g', 's_easyfun_jzbc_130g_180709', 's_easyfun_nanguatang_154g', 's_easyfun_niurougan_250g', 's_easyfun_niurougan_250g_180808', 's_easyfun_qmmb_600g', 's_easyfun_sausage_300g', 's_easyfun_suannaigan_200g', 's_easyfun_surou_180815', 's_easyfun_wc_21wancan', 's_easyfun_ximeigan_240g', 's_easyfun_yingzuidou_180g', 's_easyfun_yumixu_tea_40g', 's_easyfun_zimiquan_240g', 's_easyfun_zishuzhou_350g', 's_easys_52_qingduanshi', 's_ef_beef_spiced_100g', 's_ef_dzbtwl_300g', 's_ef_dzdqjrb_80g', 's_ef_dzhjjrc_300g', 's_ef_dzmys_150g', 's_ef_gdbwbdg_50g', 's_ef_gdbxsy_tj_25g', 's_ef_xgscrp_37g', 's_ef_ysalb_160g', 's_ef_yyfs_beef_41g', 's_ef_zuizuidong_360g', 's_gaodanbai_180g', 's_oday_dbb_both_240g', 's_oday_jxr_xl_190604', 's_oday_qiyazibinggan_256g', 's_qianka_1000k_qingliang_20190703', 's_qianka_1000k_zhongkou_20190703']
goods_df = _load_data(goods_file,goods_feature_list)
goods_df =goods_df[~goods_df['goods_slug'].isin(['goods_slug'])]

In [4]:
predict_file = '/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/predict_user_feature_data/1222/predict_user_featuere_1.csv'
predict_feature_list = ['second_id', 'birthday', 'phone_model', 'city_level', 'target_weight', 'start_weight', 'latest_weight', 'bmi', 'budget_calory', 'view_goods_count', 'add_cart_count', 'commit_order_count', 'num_paid', 'total_cost', 'record_food_count', 'record_weight_count', 'finished_sports_count', 'habbit_checkin_count', 'post_create_count', 'active_day_count', 'purpose_label', 'activate_channel_label', 'gender_label']
predict_user_df = _load_data(predict_file,predict_feature_list)
predict_user_df = predict_user_df.drop_duplicates('second_id')
predict_user_df =predict_user_df[~predict_user_df['second_id'].isin(['second_id'])]
predict_user_df['second_id'] = predict_user_df['second_id'].astype(int)

/Users/wjj/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [349]:
#predict_user_df_1 = predict_user_df.tail(11000)

In [5]:
predict_user_df = predict_user_df.apply(pd.to_numeric)
for i in goods_df.columns.tolist():
    if i == 'goods_slug':
        continue
    goods_df[i] =goods_df[i].apply(pd.to_numeric)

In [6]:
lr_1 = joblib.load('/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/model/1228/lr_1')
xgboost = joblib.load('/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/model/1228/xgboost')
xgbenc = joblib.load('/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/model/1228/xgbenc')

In [118]:
#这三个类别不平衡
#buy_and_not_buy_df = buy_and_not_buy_df[~(buy_and_not_buy_df['s_slug']=='s_ef_dzhjjrc_300g')]
#buy_and_not_buy_df = buy_and_not_buy_df[~(buy_and_not_buy_df['s_slug']=='s_coffee_190819_1_1')]
#buy_and_not_buy_df = buy_and_not_buy_df[~(buy_and_not_buy_df['s_slug']=='s_ef_dzdqjrb_80g')]

In [11]:
punish_dict = {
    's_oday_jxr_xl_190604':-0.09,
    'p_ea10days_1512g':-0.06,
    's_easyfun_blackrice_xjyz_zs_qyz':-0.13,
    's_ES_20190724':-0.1,
    's_ea_dcnx_240g':-0.07,
    's_cm25_shucai_50g':-0.02,
    's_easyfun_sausage_300g':-0.08,
    's_easyfun_heimiquan_240g':-0.03,
    's_easyfun_zimiquan_240g':-0.04,
    's_easys_52_qingduanshi':-0.06,
    's_easyfun_wc_21wancan':-0.06,
    's_oday_dbb_both_240g':-0.01,
    's_cm25_yyb_384g':-0.04,
    'p_ea_3daypttp':-0.04,
    's_cm25_yykkxin_420g_new_190103':-0.05,
    's_bh_BH20F_mini_charge':0.1,
    's_easyfun_qmmb_600g':0.08,
    's_ef_gdbxsy_tj_25g':0.08,
    's_easyfun_jzbc_130g_180709':0.09,
    's_easyfun_suannaigan_200g':0.06,
    's_ef_qyzdrwh_112g':0.4,
    's_ef_dzbtwl_300g':0.09,
    's_easyfun_mochahongdou_240g':0.11,
    's_cm25_gxmydr_300g':0.08,
    's_ef_dzdqjrb_80g':0.07,
    'p_ef_funsichi_nwlb':0.08,
    's_easyfun_chiaoat_300g':-0.04,
    's_ef_zuizuidong_360g':0.09,
    's_ef_gdbwbdg_50g':0.09,
    'p_ef_zkxsd_757g':0.09,
    's_ef_dzmys_150g':0.09,
    's_easyfun_surou_180815':0.06,
    's_easyfun_dongyingong_350g':0.08,
    's_easyfun_zc_191028':0.16,
    's_ef_llggmp_240g':0.16,
    's_NICE_service_lw':0.12,
    
}

In [12]:
def predict(row):
    user_id = int(row['second_id']) 
    out_dict={}  
    del row['second_id']
    for index_2,row_2 in goods_df.iterrows():
        s_slug = row_2['goods_slug']  
        del row_2['goods_slug']
        row_3= pd.concat([row,row_2])
        df2 = pd.DataFrame(row_3,dtype=np.float).T      
        predict_leaves = xgboost.apply(df2)
        predict_x_one_hot=xgbenc.transform(predict_leaves).toarray()
        predict_test=lr_1.predict_proba(predict_x_one_hot)[:, 1]
        if s_slug in punish_dict:
            score = predict_test[0]
            #score = predict_test[0] + punish_dict[s_slug]
        else:
            score = predict_test[0]
        out_dict.update({s_slug:score}) 
    list1= sorted(out_dict.items(),key=lambda x:x[1],reverse=True)
    list2 =[user_id,list1[0][0],list1[1][0],list1[2][0]]
    return list2

In [14]:
def apply_parallel(data_df, func):
    user_recommender_dict = Parallel(n_jobs=-1)(delayed(func)(row) for index,row in data_df.iterrows())
    user_recommender_df = DataFrame(user_recommender_dict,columns=['second_id','slug_1','slug_2','slug_3'])
    return user_recommender_df
predict_user_df_2 = predict_user_df.tail(300)
#start = time.perf_counter()
user_recommender_df =apply_parallel(predict_user_df_2,predict)
#end = time.perf_counter()
#user_recommender_df.to_csv('/Users/wjj/Desktop/jupyter/Boohee_GBDT+LR/result/1227/%s.csv'%3)
#print('%.10f'%(end - start))

In [16]:
user_recommender_df

,second_id,slug_1,slug_2,slug_3
0,23163346,s_easyfun_blackrice_xjyz_zs_qyz,s_easyfun_chiaoat_300g,s_easyfun_heimiquan_240g
1,23164053,s_easyfun_surou_180815,s_bu_BH20SW_white,s_easyfun_sausage_300g
2,23164528,s_easyfun_surou_180815,s_bu_BH20SW_white,s_easyfun_sausage_300g
3,23166666,s_easyfun_blackrice_xjyz_zs_qyz,s_easyfun_chiaoat_300g,s_easyfun_heimiquan_240g
4,23169753,s_easyfun_surou_180815,s_bu_BH20SW_white,s_easyfun_sausage_300g
5,23170120,s_bu_BH20SW_white,s_easyfun_surou_180815,s_easyfun_blackrice_xjyz_zs_qyz
6,23172358,s_bu_BH20SW_white,s_easyfun_surou_180815,s_ea_yyyxcw_420g
7,23175885,s_easyfun_surou_180815,s_ea_yyyxcw_420g,s_bu_BH20SW_white
8,23177197,s_bu_BH20SW_white,s_easyfun_surou_180815,s_ea_yyyxcw_420g
9,23178704,s_easyfun_blackrice_xjyz_zs_qyz,s_easyfun_chiaoat_300g,s_easyfun_heimiquan_240g


In [19]:
tmp_list = []
for row in user_recommender_df.itertuples():     
    for i in range(2,5):
       
        tmp_list.append(row[i])
tmp_dict = {}
for i in set(tmp_list):
    tmp_dict.update({i:tmp_list.count(i)})
tmp_list_2= sorted(tmp_dict.items(),key=lambda x:x[1],reverse=True) 
tmp_list_2

[('s_bu_BH20SW_white', 191),
 ('s_easyfun_surou_180815', 185),
 ('s_easyfun_blackrice_xjyz_zs_qyz', 101),
 ('s_ea_yyyxcw_420g', 87),
 ('s_easyfun_sausage_300g', 78),
 ('s_easyfun_chiaoat_300g', 73),
 ('s_ES_20190724', 50),
 ('s_easyfun_heimiquan_240g', 42),
 ('s_qianka_1000k_qingliang_20190703', 41),
 ('s_oday_jxr_xl_190604', 23),
 ('s_bh_BH20F_mini_charge', 12),
 ('s_easyfun_wc_21wancan', 6),
 ('s_easyfun_curryrice_160g', 3),
 ('s_easys_52_qingduanshi', 3),
 ('s_easyfun_zishuzhou_350g', 3),
 ('s_ef_dzdqjrb_80g', 2)]

# 草稿本